In [2]:
import pandas as pd

In [4]:
pd.read_csv('20200319-071433.csv')

,local date,local time,city,lat,lon,aqi,co,h,no2,p,pm10,pm25,so2,t,w,wg,o3,dew
0,2020-03-17,23:00:00,"Gramsci, Firenze, Italy",43.770586,11.271415,57,0.1,37.6,28.8,1024.1,19.0,57.0,0.7,20.5,9.0,16.0,NaN,NaN
1,2020-03-19,01:00:00,"Barrie, Ontario, Canada",44.382361,-79.702306,32,NaN,62.0,1.7,1015.8,NaN,21.0,NaN,0.5,1.0,4.0,32.0,NaN
2,2020-03-17,23:00:00,"Rebaudengo, Torino, Piemonte, Italy",45.104332,7.696423,76,0.1,72.0,NaN,1023.6,NaN,76.0,NaN,9.4,0.2,0.7,NaN,NaN
3,2020-03-17,23:00:00,"Milano Senato, Lombardia, Italy",45.470501,9.197461,97,0.1,67.3,26.6,1024.4,40.0,97.0,NaN,10.2,NaN,NaN,NaN,NaN
4,2020-03-17,23:00:00,"Arenula, Roma, Lazio, Italy",41.893267,12.475385,50,NaN,45.5,12.8,1021.6,29.0,50.0,NaN,20.0,1.5,5.0,17.5,NaN
5,2020-03-17,23:00:00,"p.le Rosmini, Trieste, Friuli Venezia Giulia, ...",45.640486,13.766081,55,NaN,8.0,NaN,1026.0,18.0,55.0,0.3,22.7,3.0,9.0,NaN,NaN
6,2020-03-17,23:00:00,"Porta San Felice, Emilia Romagna, Italy",44.500017,11.328501,68,0.1,44.0,38.9,1028.7,24.0,68.0,NaN,15.3,0.3,1.3,30.9,NaN
7,2020-03-19,05:00:00,"Ente Ferrovie, Napoli, Campania, Italy",40.853610,14.271670,85,4.6,70.0,21.3,1024.0,29.0,85.0,0.2,8.0,0.7,11.3,NaN,3.0
